# Shadow Rollout with Seldon and Ambassador

This notebook shows how you can deploy "shadow" deployments to direct traffic not only to the main Seldon Deployment but also to a shadow deployment whose reponse will be dicarded. This allows you to test new models in a production setting and with production traffic and anlalyse how they perform before putting them live.

These are useful when you want to test a new model or higher latency inference piepline (e.g., with explanation components) with production traffic but without affecting the live deployment.


## Prerequistes

You will need

 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core) running this notebook
 - A running Kubernetes cluster with kubectl authenticated
 - [Helm client](https://helm.sh/)
 - Seldon Core Python Module : `pip install seldon-core`

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

In [2]:
!kubectl create namespace seldon

namespace/seldon created


In [1]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [4]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [5]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [6]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [8]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Thu Jan 24 14:35:05 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s


NOTES:
NOTES: TODO




In [9]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Thu Jan 24 14:37:08 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.105.100.33  <none>       80:32027/TCP,443:31601/TCP     0s
seldon-core-ambassador-admin  NodePort   10.97.104.181  <none>       8877:30448/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.110.89.7    <none>       8080:30639/TCP,5000:31979/TCP  0s
seldon-core-redis             ClusterIP  10.98.98.16    <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1        1        1           0          0s
seldon-core-redis

In [10]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for deployment "seldon-core-seldon-apiserver" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


Install the Prometheus and Grafana example analytics

In [6]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false  --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Fri Mar 15 11:39:29 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     0s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           0s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.109.132.84   <none>       80/TCP        0s
grafana-prom              NodePort   10.104.112.236  <none>       80:32206/TCP  0s
prometheus-node-exporter  ClusterIP  None            <none>       9100/TCP      0s
prometheus-seldon         ClusterIP  10.96.255.219   <none>       80/TCP        0s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-ptqtw      0/1    ContainerCreating  0         0s
alertmanager-deployment-6468d9f9f7-7mtj5  0/1    Co

## Set up Port Forwards

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

**Ensure you port forward to Grafana**

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`.

In [17]:
!pygmentize model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example"
    },
    "spec": {
        "name": "production-model",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                                  ],
                "graph":
                {
                    "children": [],
                    "name": "classifier",
                    "type": "MODEL",
                    "endpoint": {
              

In [1]:
!kubectl create -f model.json

seldondeployment.machinelearning.seldon.io/example created


In [2]:
!kubectl rollout status deploy/production-model-single-7cd068f

deployment "production-model-single-7cd068f" successfully rolled out


### Get predictions

In [3]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="example",namespace="seldon")

#### REST Request

In [4]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.5400787551001015
  }
}

Response:
meta {
  puid: "n10685thmpqd8nm5og6ojc2pjd"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08497549608570185
  }
}



#### gRPC Request

In [5]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.9040190193224558
  }
}

Response:
meta {
  puid: "8hdcd21pkf3hjk83cpq7h72mbf"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11788138257822263
  }
}



## Launch Shadow

We will now create a new Seldon Deployment for our Shadow deployment with a new model `seldonio/mock_classifier_rest:1.1`. To make it a shadow of the original `example` deployment we add two annotations

```
"annotations": {
	    "seldon.io/ambassador-service-name":"example",
        "seldon.io/ambassador-shadow":"true"
	},	
```

The first says to use `example` as our service endpoint rather than the default which would be our deployment name - in this case `example-shadow`. This will ensure that this Ambassador setting will apply to the same prefix as the previous one. The second states we want to use Ambassador's shadow functionality.

In [9]:
!pygmentize shadow.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "example-shadow"
    },
    "spec": {
        "name": "shadow-model",
	"annotations": {
	    "seldon.io/ambassador-service-name":"example",
	    "seldon.io/ambassador-shadow":"true"	    
	},	
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_rest:1.1",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier"
                            }
                        ],
                        "terminationGracePeriodSeconds": 1
                    }}
                                  ],
                "graph":
                {
                    "children": [

In [7]:
!kubectl create -f shadow.json

seldondeployment.machinelearning.seldon.io/example-shadow created


In [8]:
!kubectl rollout status deploy/shadow-model-single-4c8805f

Waiting for deployment "shadow-model-single-4c8805f" rollout to finish: 0 of 1 updated replicas are available...
deployment "shadow-model-single-4c8805f" successfully rolled out


Let's send a bunch of requests to the endpoint.

In [9]:
for i in range(1000):
    r = sc.predict(gateway="ambassador",transport="rest")

Now view the analytics dashboard at http://localhost:3000 
You should see a dashboard view like below showing the two models production and shadow both receiving requests.

![shadow](shadow.png)
